In [42]:
# -*- coding: utf-8 -*-
"""
Created on Sun Feb 19 20:18:26 2017

@author: Miguel-Portatil
"""


#Library 
import urllib2
import pymongo
from bs4 import BeautifulSoup
from pymongo import MongoClient

#set the url:
url = "http://www.publico.es/sociedad/igualdad"

#Create a request
request = urllib2.Request(url)

#Now, we can create a handle which we can read out later
handle = urllib2.urlopen(request)

#Read content
content = handle.read()

soup = BeautifulSoup(content, "lxml")

newsList = soup.find_all("div", {"class":"listing-item"})

enlaces = [None]*len(newsList)
titulares = [None]*len(newsList)
autores = [None]*len(newsList)
for news in newsList:
    enlaceAux = news.find('a', href=True)
    enlaces[newsList.index(news)] = 'http://www.publico.es/' + enlaceAux['href']
    titularDiv = news.find("div", {"class":"listing-title"})
    titulares[newsList.index(news)] = titularDiv.a.string.encode('utf-8')
    autorDiv = news.find("div", {"class":"listing-signature"})
    autores[newsList.index(news)] = autorDiv.string.encode('utf-8')
    autores[newsList.index(news)] = autores[newsList.index(news)].replace(" ","")
    autores[newsList.index(news)] = autores[newsList.index(news)].replace("\n","")

htmlNoticias = [None]*len(enlaces)
textoNoticias = [None]*len(enlaces)
fechaNoticias = [None]*len(enlaces)
for enlace in enlaces:
    request = urllib2.Request(enlaces[enlaces.index(enlace)])
    handle = urllib2.urlopen(request)
    html = handle.read()
    htmlSoup = BeautifulSoup(html,"lxml")
    htmlNoticias[enlaces.index(enlace)] = htmlSoup.find("div", {"class":"article-body"})
    fechaNoticias[enlaces.index(enlace)] = htmlNoticias[enlaces.index(enlace)].find("span", {"class":"published"}).contents
    textoNoticias[enlaces.index(enlace)] = htmlNoticias[enlaces.index(enlace)].find("div", {"class":"article-text"}).contents
    
for noticia in textoNoticias:   
    textoNoticia = ""
    textoParrafoSinHtml =""
    for parrafo in noticia:
        try:
            handle = textoNoticias[textoNoticias.index(noticia)][noticia.index(parrafo)].getText()
            #print handle
            textoParrafoSinHtml = textoParrafoSinHtml + handle
        except AttributeError:
            textoParrafo = ""
        except ValueError:
            textoParrafo = ""
        #textoParrafoSinHtml = ""
        #try:
        #    for texto in textoParrafo.find('p').contents:
        #        textoParrafoSinHtml = textoParrafoSinHtml + texto
        #except AttributeError:
        #    textoParrafoSinHtml = textoParrafoSinHtml + ""
    textoNoticia =textoNoticia + textoParrafoSinHtml
    #print textoNoticia
    textoNoticias[textoNoticias.index(noticia)] = textoNoticia
    #print textoNoticias


        

cliente = MongoClient()
db = cliente.test_database
borradas = db.noticias.delete_many({})
print borradas.deleted_count
noticias = db.noticias

for enlace in enlaces:
    indice = enlaces.index(enlace)
    autor = autores[indice]
    titular = titulares[indice]
    url = enlace
    fecha = fechaNoticias[indice]
    texto = textoNoticias[indice].encode('utf-8').decode('utf-8')
    noticia = { 'author': autor, 'title':titular,
               'link':url, 'publishDate':fecha, 'text':texto}
    noticia_id = noticias.insert_one(noticia)

20
El PSOE pide reformar la ley para despatologizar la transexualidad
Facua pide a la Fiscalía que investigue la venta de los disfraces sexis para niñas
El PSOE pide al Gobierno igualdad salarial "real y efectiva" entre hombres y mujeres
"Un tercio del alumnado de los últimos cursos de primaria es víctima de acoso"
Un manifiesto pide "equilibrio de género" en el Tribunal Constitucional
Miles de personas acuden a Sol en apoyo a las ocho feministas en huelga de hambre
Un hombre asesina a su mujer en una residencia de mayores de Alicante
Una nueva 'mordaza': multa a la coordinadora de la protesta pacífica contra las violencias machistas del 7N
En EEUU, las mujeres solas no ganan ni para un alquiler bajo a causa de la brecha salarial
Brenda y María José podrán inscribir a su hijo Lennon en el Registro Civil
La última perla de un arzobispo: "Hay una patología detrás de la ideología de género"
As Encrobas: cuando la guardia civil cargaba con mosquetón contra las mujeres gallegas
Ikea ignora 

In [54]:
import re
pat = re.compile('transexual', re.I)
result = db.noticias.find({"text":{'$regex':pat}}, {'title':1});
for document in result:
    print document

{u'_id': ObjectId('58ab5f2b83523f1062a6367a'), u'title': u'El PSOE pide reformar la ley para despatologizar la transexualidad'}
{u'_id': ObjectId('58ab5f2b83523f1062a63684'), u'title': u'La \xfaltima perla de un arzobispo: "Hay una patolog\xeda detr\xe1s de la ideolog\xeda de g\xe9nero"'}


In [48]:
cursor = db.noticias.find({})

for document in cursor:
    print(document)

{u'author': u'SARAPLAZA', u'text': u'El Partido Socialista ha presentado este mi\xe9rcoles ante el Congreso de los Diputados una propuesta para la reforma de la ley 3/2007 relativa al sexo de las personas. Con esta medida se pretende despatologizar la transexualidad y avanzar hacia la autodeterminaci\xf3n de las personas.Le ley, que el pr\xf3ximo 15 de marzo cumplir\xe1 diez a\xf1os, establec\xeda en su art\xedculo 4 que el reconocimiento de la identidad de las personas transexuales deb\xeda estar acreditado por informes m\xe9dicos a la vez que se realizaba s\xf3lo si se hab\xeda producido la reasignaci\xf3n genital. \u201cCon la reforma de esta ley se pretende reconocer la autodeterminaci\xf3n de las mujeres y hombres transexuales que ya no necesitar\xe1n acreditar documentaci\xf3n m\xe9dica, ni permisos para el reconocimiento de su identidad sexual\u201d, explica a P\xfablico Carla Antonelli, diputada en la Asamblea de Madrid por el PSOE, mujer transexual y una de las impulsoras de l

In [49]:
noticia = { "author": "EFE", "title":"titular","link":"www.link.com", "publishDate":"fecha", "text":"<p id=\"m434-433-435\">Eva Romera acaba de publicar el estudio, financiado por el Centro de Estudios Andaluces, <b>\u2018Bullying, cyberbullying y dating violence\u2019.</b> A trav\xe9s de un complejo muestreo estad\xedsticos con 3.000 estudiantes de quinto y sexto de Primaria y cuatro cursos de Secundaria de 35 centros educativos p\xfablicos y privado, Romera, profesora titular del Departamento de Psicolog\xeda de la Universidad de C\xf3rdoba, ha tardado dos a\xf1os en dar resultados de esta problem\xe1tica con graves secuelas entre los m\xe1s j\xf3venes.</p>"}

In [122]:
#for parrafo in textoNoticias[0]:
parrafo =BeautifulSoup(textoNoticias[0][1].getText(), "lxml")


IndexError: string index out of range

In [111]:
parrafo = parrafo.find('p').contents
textoTotal = ""
for texto in parrafo:
    textoTotal = textoTotal + texto
    print textoTotal
print textoNoticias[0][0].getText()

El PSOE lleva a debate este martes al Pleno del Congreso una moción por la que insta al Gobierno a reducir la desigualdad salarial entre mujeres y hombres, con el fin de eliminarla. Y para ello, le pide que presente, en el plazo de seis meses, un proyecto de ley de igualdad salarial que garantice "de forma real y efectiva" la igualdad de retribución por el desempeño de un trabajo de igual valor.


AttributeError: 'NavigableString' object has no attribute 'getText'

In [19]:
autor


'5061756c6164c3ad617a'